## Individual training for Charlie

The process for each scenario includes following:
- data preprocessin
- feature extraction (PCA)
- LR
- visualization
- MLP
- hyperparameter tuning
- visualization

**importing libraries**

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt


In [3]:

scenario_one=pd.read_csv("database/Scenario_1/Scenario_1_C.csv")
scenario_two=pd.read_csv("database/Scenario_2/Scenario_2_C.csv")
scenario_three=pd.read_csv("database/Scenario_3/Scenario_3_C.csv")
scenarios=[scenario_one,scenario_two,scenario_three]

In [5]:
print(scenario_one.columns[-2],scenario_one.columns[-1])

Fermi_energy thermo_prob_norm


**Data preprocessing**

In [35]:
def columnwise_na_elimiation(data):
    """
    The method filter out all columns contains NA value in the set

    Input: pandas dataframe
    Output: non-NA containing pandas dataframe 
    """
    col_na=data.isna().sum(axis=0)
    # print(col_na)
    col_na[col_na!=0]=1
    col_na=col_na.astype(bool)
    # print(col_na)
    col_na=~col_na
    # print(col_na)
    filtered_data=data.loc[:,col_na]
    return filtered_data

nafree_sce1=columnwise_na_elimiation(scenario_one)
nafree_sce2=columnwise_na_elimiation(scenario_two)
nafree_sce3=columnwise_na_elimiation(scenario_three)
nafree_sces=[nafree_sce1,nafree_sce2,nafree_sce3]


data splitting

In [36]:
training_sets=[]
test_sets=[]
for sce in nafree_sces:
    train,test=train_test_split(sce,test_size=0.2)
    trainX=train.iloc[:,:-2]
    trainY=train.iloc[:,-2:]
    testX=test.iloc[:,:-2]
    testY=test.iloc[:,-2:]
    training_sets.append([trainX,trainY])
    test_sets.append([testX,testY])

**Feature Extraction**

In [22]:
from sklearn.decomposition import PCA

In [37]:
pca1=PCA(n_components=5)
pca2=PCA(n_components=5)
pca3=PCA(n_components=5)
pcas=[pca1,pca2,pca3]
for i,sce in enumerate(nafree_sces):
    currentPCA=pcas[i]
    currentTrainX=training_sets[i][0]
    currentTestX=test_sets[i][0]
    currentPCA.fit(currentTrainX)
    training_sets[i][0]=currentPCA.transform(currentTrainX)
    test_sets[i][0]=currentPCA.transform(currentTestX)


**Linear Regression**

In [12]:
import sys
sys.path.append("FederatedLearning-main")
import Model


In [ ]:

lr1=None
lr2=None
lr3=None
lrs=[lr1,lr2,lr3]

#Model creation for LR
for i,single_train in enumerate(training_sets):
    currModel=lrs[i]
    currX=single_train[0]
    currY=single_train[1]
    lrs[i]=Model.LinearRegression(currX.shape[1],1)


In [26]:
#Model training for LR
import Loss
import torch.optim as optim
import torch
mseLoss=Loss.MSELoss()
optimizer1=optim.SGD(lrs[0].parameters(),lr=0.01)


currOpt=optimizer1
currX=training_sets[0][0]
currX=torch.tensor(currX,dtype=torch.float32)
currY=training_sets[0][1].iloc[:,0]
currY=torch.tensor(currY,dtype=torch.float32)

for epoch in range(100):
    currOpt.zero_grad()
    predictedResult=lrs[0](currX)
    loss=mseLoss(predictedResult,currY,lrs[0])
    loss.backward()
    currOpt.step()


C:\Users\kenan\anaconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([34844])) that is different to the input size (torch.Size([34844, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [40]:
from sklearn.metrics import mean_squared_error
curr=lrs[0]
testX=torch.tensor(test_sets[0][0],dtype=torch.float32)

mean_squared_error(test_sets[0][1].iloc[:,0],(curr(testX)).detach().numpy())
print(testX)
print(curr.parameters())

tensor([[-1.9377e+01, -3.2975e+00, -5.3702e-01, -6.2125e-01,  3.2729e-01],
        [ 2.9633e+00, -1.4231e+00, -4.7729e-03, -2.4112e-01, -6.3294e-02],
        [-3.4282e+01, -8.9451e-01, -6.8482e-01, -1.1097e-01,  5.4800e-02],
        ...,
        [-1.2387e+01,  3.0937e+00, -9.5303e-01, -6.2489e-02,  1.6058e-01],
        [-6.0099e+00, -3.8075e+00, -4.6432e-01,  9.0560e-02, -2.6995e-02],
        [-2.2746e+01, -5.1098e-01,  1.8044e-01,  1.5945e-01, -2.0260e-01]])


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').